In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
import re
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import re
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
data = pd.read_csv('processed_df.csv')

**ВЫКИНЕМ НАНЫ**

In [3]:
data_no_nans = data.copy()
data_no_nans = data_no_nans.dropna()
data_no_nans.isna().sum()

Maruti                  0
Skoda                   0
Honda                   0
Hyundai                 0
Toyota                  0
Ford                    0
Renault                 0
Mahindra                0
Tata                    0
Chevrolet               0
Fiat                    0
Datsun                  0
Jeep                    0
Mercedes-Benz           0
Mitsubishi              0
Audi                    0
Volkswagen              0
BMW                     0
Nissan                  0
Lexus                   0
Jaguar                  0
Land                    0
MG                      0
Volvo                   0
Daewoo                  0
Kia                     0
Force                   0
Ambassador              0
Ashok                   0
Isuzu                   0
Opel                    0
year                    0
selling_price           0
km_driven               0
Diesel                  0
Individual              0
Dealer                  0
Manual                  0
First Owner 

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЕ И ТЕСТОВЫЕ МАССИВЫ**

In [4]:
train_data_no_nans = data_no_nans.sample(frac=0.8)
test_data_no_nans = data_no_nans.iloc[~data_no_nans.index.isin(train_data_no_nans.index), :]

In [5]:
features_train = train_data_no_nans.iloc[:, train_data_no_nans.columns != 'selling_price']
responses_train = train_data_no_nans['selling_price'].to_frame(name='selling_price')
features_test = test_data_no_nans.iloc[:, test_data_no_nans.columns != 'selling_price']
responses_test = test_data_no_nans['selling_price'].to_frame(name='selling_price')

**НОРМИРУЕМ ДАННЫЕ**

In [6]:
scaller = StandardScaler()
x_train = scaller.fit_transform(features_train)
y_train = scaller.fit_transform(responses_train)
x_test = scaller.fit_transform(features_test)
y_test = scaller.fit_transform(responses_test)

# **ОБУЧИМ МОДЕЛЬ НА ТЕХ ДАННЫХ, ЧТО ЕСТЬ**

**ОБУЧИМ МОДЕЛЬ С ЛИНЕАРИЗАЦИЕЙ КОЭФФИЦИЕНТОВ L1 LASSO**

In [7]:
l1_regr = Lasso(1e-5, max_iter=10 ** 4)
l1_regr.fit(x_train, y_train)
l1_regr.score(x_train, y_train)

0.8480561610118478

In [8]:
l1_regr.coef_

array([ 4.64033179e-02, -1.31207139e-02, -0.00000000e+00,  7.83571266e-04,
        8.33678842e-02, -2.58593835e-03, -1.88444385e-03, -1.77419004e-03,
       -4.25857708e-02, -9.39978760e-03, -7.21433909e-03, -1.16560669e-02,
        5.21018227e-02,  1.18317607e-01,  1.36033581e-02,  8.64647833e-02,
       -2.41312979e-02,  4.09372368e-01, -1.40355668e-03,  2.76104689e-01,
        1.59576471e-01,  9.13575616e-02,  8.01006495e-03,  9.82306647e-02,
        1.07543533e-02,  1.62576382e-02, -0.00000000e+00,  1.10741249e-02,
        0.00000000e+00,  2.18555302e-02,  5.83907312e-03,  2.06462078e-01,
       -6.52076298e-02, -1.72339389e-03, -5.21332121e-03,  3.24250549e-02,
       -3.57972605e-02, -1.39501425e+00, -1.31926427e+00, -7.21713534e-01,
       -4.21392196e-01, -2.02125361e-02,  4.42860949e-02,  2.66154013e-01,
        7.18828893e-03, -7.42144902e-02, -1.52735586e-02])

In [9]:
l1_regr.score(x_test, y_test)

0.826006352964241

**КАК ВИДНО НА ТЕСТОВОЙ ВЫБОРКЕ РЕЗУЛЬТАТ ДАЖЕ ЛУЧШЕ.**

**МФ ПОЛУЧИЛИ РЕЗУЛЬТАТ ПО МЕТРИКЕ R2**

**ПОПРОБУЕМ ОЦЕНИТЬ С ПОМОЩЬЮ МЕТРИКИ MAE**

In [10]:
y_train_predict = l1_regr.predict(x_train)
y_test_predict = l1_regr.predict(x_test)
loss_train = mean_absolute_error(y_train, y_train_predict)
loss_test = mean_absolute_error(y_test, y_test_predict)

In [11]:
print(loss_train)
print(loss_test)

0.19992348720534817
0.22522308484407072


**ПОСМОТРИМ СКОЛЬКО ЭТО В ДЕНЕЖНОМ ЭКВИВАЛЕНТЕ С ПОМОЩЬЮ ИНВЕРСИИ НОРМАЛИЗАЦИИ**

In [12]:
print(scaller.inverse_transform([[loss_train]]))
print(scaller.inverse_transform([[loss_test]]))

[[759890.71458156]]
[[777582.05645409]]


**ОЦЕНИМ С ПОМОЩЬЮ МЕТРИКИ MAPE**

In [13]:
loss_train_mape = mean_absolute_percentage_error(y_train, y_train_predict)
loss_test_mape = mean_absolute_percentage_error(y_test, y_test_predict)

In [14]:
print(loss_train_mape)
print(loss_test_mape)

1.4390470148565642
3.1997512701978725


**ОПЯТЬ ЗАМЕЧАЕМ, ЧТО НА ТЕСТОВЫХ ДАННЫХ РЕЗУЛЬТАТ ЛУЧШЕ**

# **ДОБАВИМ ПОЛИНОМ 9 СТЕПЕНИ ПО ПРИЗНАКАМ ТИП ТОПЛИВА И ОБЬЕМ ДВИГАТЕЛЯ**

**В РЕЗУЛЬТАТЕ АНАЛИЗА ДАННЫХ (ФАЙЛ processing_nes.ipynb) МЫ ВЫЯСНИЛИ, ЧТО ОБЬЕМ ДВИГАТЕЛЯ НАХОДИТСЯ В НЕКОТОРОЙ КОРЕЛЯЦИИ ОТ ТИПА ТОПЛИВА. ПОСТРОИВ ПОЛИНОМИАЛЬНУЮ МОДЕЛЬ, ОБУЧИВ ЕЕ И ПОСЧИТАВ ПОТЕРИ С КРОСС ВАЛИДАЦИЕЙ МЫ ПОНЯЛИ, ЧТО ДЛЯ ДВУХ ЭТИХ ДВУХ ПРИЗНАКОВ С ПОЛИНОМОМ СТЕПЕНЬЮ 9 (ФАЙЛ analyse.ipynb) МЫ МОЖЕМ УЛУЧШИТЬ МОДЕЛЬ. ПОПРОБУЕМ ЕЕ УЛУЧШИТЬ ДОБАВИВ ЭТИ ПРИЗНАКИ**

**ВОЗЬМЕМ ФРЕЙМ БЕЗ НАНОВ И СКАЛЯРИЗУЕМ**

In [15]:
scaller_poly = StandardScaler()
x_poly = scaller_poly.fit_transform(data_no_nans[data_no_nans.columns.drop('selling_price')])
y_poly = scaller_poly.fit_transform(data_no_nans[['selling_price']])

In [16]:
scaled_df = pd.DataFrame(data=np.hstack((y_poly, x_poly)), columns=['selling_price'] + list(data_no_nans.columns.drop('selling_price')))
scaled_df

,selling_price,Maruti,Skoda,Honda,Hyundai,Toyota,Ford,Renault,Mahindra,Tata,...,First Owner,Second Owner,Third Owner,Fourth & Above Owner,milage,engine,max_power,torque,RPM_min,seats
0,-0.237694,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,1.021040,-0.418993,-0.495635,0.081953,-0.617447,-0.436655
1,-0.339271,-0.651318,8.627996,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,-1.385243,1.702822,-0.263667,-0.144741,0.440338,0.077846,0.345081,0.601426,-1.065356,-0.436655
2,-0.608449,-0.651318,-0.115902,3.979562,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,-1.385243,-0.587260,3.792666,-0.144741,-0.443564,0.075858,-0.381717,-0.485485,0.009625,-0.436655
3,-0.523379,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,0.918261,-0.124865,-0.039962,0.337534,-0.841402,-0.436655
4,-0.644001,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,-0.854681,-0.319626,-0.091225,-0.587301,1.622096,-0.436655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841,-0.402756,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,-0.238005,-0.520348,-0.243591,-0.578644,1.174187,-0.436655
7842,-0.637652,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,-1.385243,-0.587260,-0.263667,6.908901,-0.674817,0.067909,0.529629,0.473289,-0.707029,-0.436655
7843,-0.324034,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,-0.032447,-0.418993,-0.498483,0.081953,-0.617447,-0.436655
7844,-0.440848,-0.651318,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,3.145985,...,0.721895,-0.587260,-0.263667,-0.144741,1.064721,-0.124865,-0.609553,-0.350941,-0.796611,-0.436655


**СОЗДАДИМ ПОЛИНОМ 9Й СТЕПЕНИ ДЛЯ ДВИГАТЕЛЯ И ДИЗЕЛЯ**

In [17]:
poly_obj = PolynomialFeatures(7)
engine_diesel_poly = poly_obj.fit_transform(scaled_df[['Diesel', 'engine']])
engine_diesel_poly.shape

(7846, 36)

**УБЕРЕМ ПЕРВЫЕ ТРИ СТОЛБЦА, ПЕРВЫЙ ПОТОМУ ЧТО ЕДИНИЧНЫЙ ДЛЯ СВОБОНОГО ВЕСА, ВТОРЫЕ ДВА ПОТОМУ ЧТО ПОВТОРЕНИЕ ТЕХ, ЧТО УЖЕ ЕСТЬ В ДАТАСЕТЕ, ЗАПИШЕМ ВО ФРЕЙМ**

In [18]:
engine_diesel_poly_df = pd.DataFrame(data=engine_diesel_poly[:, 3:], columns=[f'Dies*Engine{i + 1}' for i in range(engine_diesel_poly.shape[1] - 3)])

**СОБЕРЕМ ПОЛИНОМ С ДАТАСЕТОМ**

In [19]:
poly_prepared = pd.concat([scaled_df, engine_diesel_poly_df], axis=1)

**РАЗДЕЛИМ ВЕСЬ СКАЛЯРИЗОВАННЫЙ ДАТАСЕТ НА ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ**

In [20]:
poly_prepared_train = poly_prepared.sample(frac=0.8, random_state=0).sort_index()
poly_prepared_test = poly_prepared.iloc[~scaled_df.index.isin(poly_prepared_train.index)].sort_index()

**НАЧНЕМ ОБУЧАТЬ МОДЕЛЬ С L1/L2 НА ТРЕНИРОВОЧНОМ ДАТАСЕТЕ**

In [21]:
regr_poly = Ridge(alpha=0.0001, max_iter=10 ** 4)
regr_poly.fit(poly_prepared_train.iloc[:, 1:], poly_prepared_train.iloc[:, 0])

Ridge(alpha=0.0001, max_iter=10000)

In [22]:
regr_poly.coef_

array([-3.00308581e-02, -1.62869805e-02, -2.96341668e-02, -5.88115790e-02,
        4.09414711e-02, -3.73786131e-02, -3.49581162e-02, -3.83135220e-02,
       -8.24890993e-02, -3.86350279e-02, -1.99116696e-02, -2.61835045e-02,
        1.87878226e-02,  1.12371785e-01,  6.35399695e-03,  7.41730940e-02,
       -4.30919712e-02,  3.75621764e-01, -1.95223473e-02,  2.51302721e-01,
        1.58270994e-01,  7.35342577e-02,  7.21302824e-03,  9.83840899e-02,
        6.19669856e-03,  1.26722834e-02, -4.20176035e-03,  1.31691887e-02,
       -5.71538763e-26,  1.90424539e-02,  5.35868878e-03,  2.16902115e-01,
       -6.62337239e-02,  8.04581839e-03,  2.11065266e-03,  3.89263889e-02,
       -2.99964823e-02, -1.51070685e+00, -1.42565536e+00, -7.93000524e-01,
       -4.53319964e-01, -3.45916523e-02, -2.36121294e-02,  3.29962334e-01,
        1.24054469e-02, -5.93339283e-02, -6.78569903e-03, -1.60352845e-03,
        1.71104783e-03, -6.35148975e-02,  8.36540942e-03, -2.39532838e-02,
        5.43219211e-04, -

**ОЦЕНИМ ПОТЕРИ ПО МЕТРИКЕ MAE**

In [23]:
y_train_predict = regr_poly.predict(poly_prepared_train.iloc[:, 1:])
y_test_predict = regr_poly.predict(poly_prepared_test.iloc[:, 1:])
loss_train = mean_absolute_error(poly_prepared_train.iloc[:, 0], y_train_predict)
loss_test = mean_absolute_error(poly_prepared_test.iloc[:, 0], y_test_predict)

In [24]:
print(loss_train)
print(loss_test)

0.19468645364990594
0.19743002855917136


In [25]:
print(scaller_poly.inverse_transform([[loss_train]]))
print(scaller_poly.inverse_transform([[loss_test]]))

[[790535.29823111]]
[[792696.08909151]]


**ОЦЕНИМ ПОТЕРИ ПО МЕТРИКЕ MAPE**

In [26]:
loss_train_mape = mean_absolute_percentage_error(poly_prepared_train.iloc[:, 0], y_train_predict)
loss_test_mape = mean_absolute_percentage_error(poly_prepared_test.iloc[:, 0], y_test_predict)
print(loss_train_mape)
print(loss_test_mape)

1.1750114299191143
1.3181118370749954


**КАК ВИДНО РЕЗУЛЬТАТЫ ЧУУУУУУУТЬ ЧУТЬ УЛУЧШИЛИСЬ, ПОПРОБУЕМ ЕЩЕ УЛУЧШИТЬ МОДЕЛЬ**

# **МОЩНОСТЬ, ТИП ДВИГАТЕЛЯ И КРУТЯЩИЙ МОМЕНТ**

In [27]:
data = pd.read_csv('processed_df.csv')
data = data.dropna()

**ВИДЕЛИМ ПРИЗНАКИ И ОТКЛИКИ**

In [28]:
x = data.loc[:, data.columns != 'selling_price']
y = data[['selling_price']]

**СКАЛЯРИЗУЕМ ПРИЗНАКИ И ОТКЛИКИ**

In [29]:
scall = StandardScaler()
x_scld = scall.fit_transform(x)
y_scld = scall.fit_transform(y)

In [30]:
features_df = pd.DataFrame(data=x_scld, columns=x.columns)

**СОЗДАДИМ ПОЛИНОМ 3Й СТЕПЕНИ ПО ТИПУ ТОПЛИВА, МОЩНОСТИ И КРУТЯЩЕМУ МОМЕНТУ**

In [31]:
poly3 = PolynomialFeatures(3)
x_poly = poly3.fit_transform(features_df.loc[:, ['Diesel', 'max_power', 'torque']])[:, 4:]

**СОЗДАДИМ ОБЩИЙ ФРЕЙМ ПРИЗНАКОВ**

In [32]:
poly_df = pd.DataFrame(data=x_poly, columns=[f'Dies*Pow*Tor{i + 1}' for i in range(x_poly.shape[1])])
resp_df = pd.DataFrame(data=y_scld, columns=['selling_price'])

In [33]:
poly_df = pd.concat([resp_df, features_df, poly_df], axis=1)

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ ДАТАСЕТЫ**

In [34]:
train = poly_df.sample(frac=0.8, random_state=0)
test = poly_df.iloc[~poly_df.index.isin(train.index), :]

**ПОПРОБУЕМ ОБУЧИТЬ МОДЕЛЬ С L1 И ПОСМОТРИМ НА РЕЗУЛЬТАТ РАЗНЫМИ МЕТРИКАМИ**

In [35]:
lin_regr = Ridge(alpha=0.000001, max_iter=10000)
lin_regr.fit(train.loc[:, train.columns != 'selling_price'], train['selling_price'])

Ridge(alpha=1e-06, max_iter=10000)

In [36]:
lin_regr.coef_

array([-2.34526389e-02, -2.31339282e-02, -4.61020530e-03, -4.50053976e-02,
        6.14467139e-02, -2.98681021e-02, -3.42772473e-02, -6.38405761e-02,
       -9.10267774e-02, -4.80038933e-02, -1.80344653e-02, -3.27577186e-02,
        2.61608293e-02,  7.68005374e-02, -4.01013336e-05,  5.81766325e-02,
       -4.21268934e-02,  3.53846058e-01, -1.98371312e-02,  3.22928050e-01,
        1.20241584e-01,  6.31244434e-02,  2.10448495e-03,  6.78589623e-02,
        8.73118359e-03,  1.40942330e-02, -7.02010199e-03,  4.21872133e-03,
       -8.75192162e-24,  1.70821455e-02,  5.17419602e-03,  2.43091362e-01,
       -6.69349836e-02, -1.04031108e-02, -5.50730052e-03,  3.65877283e-02,
       -3.13922972e-02, -1.29349801e+00, -1.22555540e+00, -6.82321178e-01,
       -3.88350799e-01, -6.39523592e-02,  3.06904131e-02,  9.95148650e-03,
        1.71126401e-01,  6.88452741e-03, -1.26534212e-02,  2.07307207e-03,
       -6.45222032e-02, -6.86907977e-02, -8.01934560e-02,  3.22408573e-01,
       -5.16685213e-02, -

In [37]:
y_predict_train = lin_regr.predict(train.loc[:, train.columns != 'selling_price'])
y_predict_test = lin_regr.predict(test.loc[:, test.columns != 'selling_price'])
mae_train = mean_absolute_error(train['selling_price'], y_predict_train)
mae_test = mean_absolute_error(test['selling_price'], y_predict_test)

In [38]:
print(mae_train)
print(mae_test)

0.19456877996664038
0.19888102918564624


In [42]:
print(scall.inverse_transform([[mae_train]]))
print(scall.inverse_transform([[mae_test]]))

[[790442.62052858]]
[[793838.87140867]]
